# 1vb4

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1454]
        t2 = t[:, 1455:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

,fnames,score,omega,phi,prou
,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1212.62,"[-0.40211, 0.556162, 0.727318]",99.7607,"[33.9787, 31.7831, -5.51806]"
2,1/complex.2.pdb,1183.73,"[-0.315936, 0.200409, -0.927373]",177.03,"[-19.7002, 58.6909, 19.3948]"
3,1/complex.3.pdb,1135.3,"[0.00306483, -0.475415, -0.879756]",75.2811,"[0.242413, 4.8558, -2.6232]"
4,1/complex.4.pdb,1133.74,"[0.0539249, 0.0942512, -0.994087]",176.792,"[22.8572, 60.7278, 6.99761]"
5,1/complex.5.pdb,1122.77,"[-0.367839, -0.351645, 0.860837]",167.239,"[22.0496, 57.3307, 32.841]"
6,1/complex.6.pdb,1114.58,"[0.523125, 0.325591, -0.787611]",162.668,"[22.4456, 57.3706, 38.6247]"
7,1/complex.7.pdb,1111.9,"[-0.376166, 0.58079, 0.721929]",114.399,"[32.0743, 29.743, -7.21562]"
8,1/complex.8.pdb,1108.98,"[-0.261413, -0.320513, 0.910459]",172.943,"[21.9754, 56.3148, 26.1344]"
9,1/complex.9.pdb,1107.44,"[0.0334642, -0.88584, -0.462782]",176.776,"[19.4078, -6.87396, 14.5613]"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)